In [2]:
import pandas as pd

productSrc1 = pd.read_excel('ETLDataSource1.xlsx', sheet_name='productSource1')
orderSrc1 = pd.read_excel('ETLDataSource1.xlsx', sheet_name='orderSource1')
stateAbbriv = pd.read_excel('ETLDataSource1.xlsx', sheet_name='StateLookup')

productSrc2 = pd.read_excel('ETLDataSource2.xlsx', sheet_name='productSource2')
orderSrc2 = pd.read_excel('ETLDataSource2.xlsx', sheet_name='orderSource2')

In [3]:
joinedSrc1 = pd.merge(productSrc1, orderSrc1, on='OrderID', how='inner')

joinedSrc1['CustomerState'] = joinedSrc1['CustomerState'].map(stateAbbriv.set_index('Abbreviation')['State'].to_dict())

joinedSrc1.reindex(sorted(joinedSrc1.columns), axis=1)

joinedSrc2 = pd.merge(productSrc2, orderSrc2, on='OrderID', how='inner')

joinedSrc2['OrderID'] = joinedSrc2['OrderID'].replace('A', '', regex=True)

joinedSrc2['TotalDiscount'] = joinedSrc2['FullPrice'].astype(float) - joinedSrc2['ExtendedPrice'].astype(float)

joinedSrc2['CustomerName'] = joinedSrc2['CustomerFirstName'].astype(str) + ' ' + joinedSrc2['CustomerLastName'].astype(str)

joinedSrc2 = joinedSrc2.drop(['CustomerFirstName','CustomerLastName'], axis=1)

joinedSrc2.reindex(sorted(joinedSrc2.columns), axis=1)

new = pd.concat([joinedSrc1, joinedSrc2])

desired_index = ['OrderID', 'OrderDate', 'CustomerName', 'CustomerCity', 'CustomerState', 'CustomerStatus', 'ProductID', 'Product', 'UnitPrice', 'Quantity', 'FullPrice', 'ExtendedPrice', 'Discount', 'TotalDiscount']

new = new.reindex(desired_index, axis=1)

new.to_excel('PDOutput.xlsx', index=False)

In [148]:
print(joinedSrc2.head())

   ProductID                 Product  UnitPrice  Quantity  Discount  \
0          2                   Chang       15.2        50       0.2   
1         32      Mascarpone Fabioli       25.6         6       0.2   
2          5  Chef Anton's Gumbo Mix       17.0        65       0.2   
3         21     Sir Rodney's Scones        8.0        10       0.0   
4         37              Gravad lax       20.8         1       0.0   

   FullPrice  ExtendedPrice  TotalDiscount OrderID CustomerCity  \
0      760.0         608.00         152.00   10258  Kansas City   
1      153.6         122.88          30.72   10258  Kansas City   
2     1105.0         884.00         221.00   10258  Kansas City   
3       80.0          80.00           0.00   10259   Greensboro   
4       20.8          20.80           0.00   10259   Greensboro   

    CustomerState  CustomerStatus  OrderDate         CustomerName  
0          Kansas               2 2011-01-11  Milagros Fehrenbach  
1          Kansas               2 